# *k*-NN Speedup in `scikit-learn`
`scikit-learn` contains two options to speed up neighbour retrieval for *k*-NN, `kd_tree` and `ball_tree`.  
There is also the option to use brute force search, i.e. linear search across all training data.  
Here we test the performance of these three options on four datasets from the UCI repository.  
The three methods return the same nearest neighbours so we only look at retrieval times.  

To run this notebook you will need to download the three data files and the `.py` file that loads the data:  
- `kNNDataLoader.py`
- `CC_default.csv`
- `HTRU_2.csv`
- `shuttle.csv`
- `letter-recognition.csv`  

These are all available in the git-hub repository. 

We test retrieval times under two scenarios, 10-fold cross validation and 2-fold cross validation.  
The two speed up alternatives have a preprocessing overhead where the trees are built, this overhead will have a bigger impact in 10-fold cross vaildation testing. 

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import time
import statistics
from sklearn import metrics
import kNNDataLoader
import matplotlib.pyplot as plt

In [ ]:
Name_arr, X_dir, y_dir = kNNDataLoader.data_loader()
Name_arr

## Analysis
Note: the evaluations entail cross-validation testing so those cells take a few minutes to run.

In [ ]:
# Listing the datasets in this order produces nicer graphs. 
Name_arr = ['HTRU', 'Shuttle','Letter','Credit']
methods = ['brute','kd_tree', 'ball_tree', ]

In [ ]:
md10 = {}
for m in methods:
    print(m)
    rd = {}
    for ds in Name_arr:
        print(ds)
        X = X_dir[ds]
        y = y_dir[ds]
        kNN = KNeighborsClassifier(n_neighbors=50, algorithm = m)
        t_start = time.time()
        scores = cross_val_score(kNN, X, y, cv=10)
        t = time.time()-t_start
        rd[ds]=t
        print(t)
    md10[m]=rd

In [ ]:
md2 = {}
for m in methods:
    print(m)
    rd = {}
    for ds in Name_arr:
        print(ds)
        X = X_dir[ds]
        y = y_dir[ds]
        kNN = KNeighborsClassifier(n_neighbors=50, algorithm = m)
        t_start = time.time()
        scores = cross_val_score(kNN, X, y, cv=2)
        t = time.time()-t_start
        rd[ds]=t
        print(t)
    md2[m]=rd

In [ ]:
xv2res = pd.DataFrame(md2)
xv10res = pd.DataFrame(md10)
xv10res

In [ ]:
a = xv2res.values
xv2res.iloc[:,0:3] = a[:,0:3]/a[:,0,None]
a = xv10res.values
xv10res.iloc[:,0:3] = a[:,0:3]/a[:,0,None]
xv10res

In [ ]:
xv2res = xv2res.sort_values('ball_tree')
xv10res = xv10res.sort_values('ball_tree')

In [ ]:
ax = xv2res.T.plot(kind ='bar')
ax.set_ylabel("Processing Time")
ax.set_title("Processing Time 2-fold x-val")
ax.set_ylim(0,3.5)
ax.grid('on', which='major', axis='y')

In [ ]:
ax = xv10res.T.plot(kind ='bar')
ax.set_ylabel("Processing Time")
ax.set_title("Processing Time 10-fold x-val")
ax.set_ylim(0,3.5)
ax.grid('on', which='major', axis='y')